# Flight Comptuer

We recorded a lot of data about the flight comptuer itself during the flight.


## System Power Comumption

Power used by each subsystem over time

In [ ]:
from numpy import subtract, divide, multiply, median, std, var, average, arange, absolute
from scipy.signal import butter, lfilter, freqz
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('png', 'pdf')
import launch

def lowpass(port):
    # Filter requirements.
    order = 6
    fs = launch.rnhp.fs   # sample rate, Hz
    cutoff = 10       # desired cutoff frequency of the filter, Hz
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq

    # Get the filter coefficients so we can check its frequency response.
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    return lfilter(b, a, port)

port_1_filter = lowpass(launch.rnhp.port_1)
port_2_filter = lowpass(launch.rnhp.port_2)
port_3_filter = lowpass(launch.rnhp.port_3)
port_4_filter = lowpass(launch.rnhp.port_4)
port_6_filter = lowpass(launch.rnhp.port_6)
port_7_filter = lowpass(launch.rnhp.port_7)
port_8_filter = lowpass(launch.rnhp.port_8)

In [ ]:
fig, ax1 = plt.subplots(figsize=(16,6))
plt.title(r"Avionics System Current Consumption")
plt.xlabel(r"Mission Elapsed Time [s]")
plt.ylabel(r"Current [A]")

ax1.plot(launch.rnhp.time[100:-100], port_1_filter[100:-100], alpha=0.75, label="Flight Comptuer")
ax1.plot(launch.rnhp.time[100:-100], port_8_filter[100:-100], alpha=0.75, label="IMU")
ax1.plot(launch.rnhp.time[100:-100], port_3_filter[100:-100], alpha=0.75, label="GPS")
ax1.plot(launch.rnhp.time[100:-100], port_7_filter[100:-100], alpha=0.75, label="Roll Control Servo Actuator")
ax1.plot(launch.rnhp.time[100:-100], port_6_filter[100:-100], alpha=0.75, label="WiFi Power Amp")

plt.xlim([-5,42])
plt.ylim([0,1.5])
plt.legend(loc=2)
plt.show()

The roll control system comes on at $t=0$ and turns intself off at $t=32$. This does appear to mess with the power draw of the whole system. There is a mysterious jump in the WiFi power draw at $~t=26$.

In [ ]:
fig, ax1 = plt.subplots(figsize=(16,6))
plt.title(r"Avionics System Voltage")
plt.xlabel(r"Mission Elapsed Time [s]")
plt.ylabel(r"Voltage [V]")

ax1.plot(launch.rnhh.time, launch.rnhh.voltage, alpha=0.75, label="System Voltage")

plt.xlim([-5,42])
plt.ylim([14,18])
#ax1.legend(loc=2)
plt.show()

Not much to see here. Two small transients that seem to match the large current draw spikes in the system power.

In [ ]:
print """The average power draw during the flight was %0.2f Watts (%0.2f Amps at %0.1f Volts)
""" % (average(launch.rnhh.voltage) * -average(launch.rnhh.current), -average(launch.rnhh.current), average(launch.rnhh.voltage))

That's a lot of power! Our system battery is a 4s stack of liPo's rated for ~60 Wh. That's about two hours of autonomous flight. We have umbilical power until the last few minutes of the countdown.

## RasPi Computers

In addition to the main flight computer, we had two Raspberry Pi single-board computers as video recorder/processers. These also took power from the system power supply:

In [ ]:
fig, ax1 = plt.subplots(figsize=(16,6))
plt.title(r"Avionics System Current Consumption")
plt.xlabel(r"Mission Elapsed Time [s]")
plt.ylabel(r"Current [A]")

ax1.plot(launch.rnhp.time[100:-100], port_2_filter[100:-100], alpha=0.75, label="RasPi 1")
ax1.plot(launch.rnhp.time[100:-100], port_4_filter[100:-100], alpha=0.75, label="RasPi 2")

plt.xlim([-5,42])
plt.ylim([0,1.3])
plt.legend(loc=2)
plt.show()

Due to a camera failure just before flight, we flew with RasPi 1 powered off.

## Missing Packets

One potential question to ask is there any correlation with the power system and missing data packets. Here is the total power, adding up the current reported from each subsystem, both raw and low pass filtered.

There are some spikes, do they line up at all with the missing data?

In [ ]:
total_power = launch.rnhp.port_1 + launch.rnhp.port_2 + launch.rnhp.port_3 + launch.rnhp.port_4 + launch.rnhp.port_6 + launch.rnhp.port_7 + launch.rnhp.port_8

# Filter requirements.
order = 6
fs = launch.rnhp.fs
cutoff = 15       # desired cutoff frequency of the filter, Hz
nyq = 0.5 * fs
normal_cutoff = cutoff / nyq

# Get the filter coefficients so we can check its frequency response.
b, a = butter(order, normal_cutoff, btype='low', analog=False)
total_power_filter = lfilter(b, a, total_power)

fig, ax1 = plt.subplots(figsize=(16,6))
plt.title(r"Avionics System Current Consumption")
plt.xlabel(r"Mission Elapsed Time [s]")
plt.ylabel(r"Current [A]")
ax1.plot(launch.rnhp.time, total_power, 'k-', alpha=0.12, label="Total Power Raw")
ax1.plot(launch.rnhp.time[100:-100], total_power_filter[100:-100], alpha=0.8, label="Total Power Filtered")

for t in launch.seqe.time:
    plt.plot((t, t), (-10, 10), 'k-', alpha=0.7)
plt.plot((-5, -5), (-10, 10), 'k-', alpha=0.7, label="Data Gaps")

plt.xlim([-5,42])
plt.ylim([0,5.5])
plt.legend(loc=2)
plt.show()

Nothing obvious in total power, the apparent missing data right at liftoff upon further investigation is spurious and not a real data gap.

In [ ]:
fig, ax1 = plt.subplots(figsize=(16,6))
plt.title(r"Avionics System Current Consumption")
plt.xlabel(r"Mission Elapsed Time [s]")
plt.ylabel(r"Current [A]")

ax1.plot(launch.rnhp.time[100:-100], port_1_filter[100:-100], alpha=0.75, label="Flight Comptuer")
ax1.plot(launch.rnhp.time[100:-100], port_8_filter[100:-100], alpha=0.75, label="IMU")
ax1.plot(launch.rnhp.time[100:-100], port_3_filter[100:-100], alpha=0.75, label="GPS")
ax1.plot(launch.rnhp.time[100:-100], port_7_filter[100:-100], alpha=0.75, label="Roll Control Servo Actuator")
ax1.plot(launch.rnhp.time[100:-100], port_6_filter[100:-100], alpha=0.75, label="WiFi Power Amp")
ax1.plot(launch.rnhp.time[100:-100], port_4_filter[100:-100], alpha=0.75, label="RasPi 2")

for t in launch.seqe.time[1:]:
    plt.plot((t, t), (-10, 10), 'k-', alpha=0.7)
plt.plot((-5, -5), (-10, 10), 'k-', alpha=0.7, label="Data Gaps")

plt.xlim([-5,42])
plt.ylim([0,1.9])
plt.legend(loc=2)
plt.show()

A mildly plausible correlation in Flight Comptuer power at $t=34.72$. Nothing else stands out.